In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/monkeypox-skin-lesion-dataset/Monkeypox_Dataset_metadata.csv
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M34_02_07.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M53_01_02.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M50_03_04.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M34_04_07.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M34_01_09.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M34_03_13.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M17_02_10.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M50_03_06.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M17_01_13.jpg
/kaggle/input/monkeypox-skin-lesion-dataset/Fold1/Fold1/Fold1/Val/Monkeypox/M34_02_10.jpg
/kaggle/input/monkeypox-s

In [2]:
pip install torch torchvision transformers


Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Subset, TensorDataset
from transformers import SwinForImageClassification
import os

from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np
import cv2

# Paths and parameters
folder_path = "/kaggle/input/monkeypox-skin-lesion-dataset/Augmented Images/Augmented Images"
subfolders = ['Monkeypox_augmented', 'Others_augmented']
image_size = (224, 224)
batch_size = 16
epochs = 30
learning_rate = 0.0001

# Image transformations
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset
data = []
labels = []
label_map = {subfolder: idx for idx, subfolder in enumerate(subfolders)}

for subfolder in subfolders:
    subfolder_path = os.path.join(folder_path, subfolder)
    for filename in os.listdir(subfolder_path):
        img_path = os.path.join(subfolder_path, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, image_size)
            data.append(img)
            labels.append(label_map[subfolder])

data = np.array(data).astype("float32")
labels = np.array(labels)

# Create PyTorch datasets
data = torch.tensor(data).permute(0, 3, 1, 2) / 255.0  # Normalize
labels = torch.tensor(labels)

# Create a TensorDataset
dataset = TensorDataset(data, labels)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_model = SwinForImageClassification.from_pretrained(
    "microsoft/swin-base-patch4-window7-224", 
    num_labels=2,
    ignore_mismatched_sizes=True
).to(device)


# Define custom Swin Transformer classifier
class SwinClassifier(nn.Module):
    def __init__(self, swin_model):
        super(SwinClassifier, self).__init__()
        self.swin_model = swin_model

    def forward(self, x):
        return self.swin_model(pixel_values=x).logits

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_preds, train_targets = [], []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            pixel_values = images

            optimizer.zero_grad()
            outputs = model(pixel_values)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            train_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_targets.extend(labels.cpu().numpy())

        train_loss /= len(train_loader.dataset)
        train_accuracy = accuracy_score(train_targets, train_preds)

        model.eval()
        val_loss = 0.0
        val_preds, val_targets = [], []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                pixel_values = images

                outputs = model(pixel_values)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

                val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_targets.extend(labels.cpu().numpy())

        val_loss /= len(val_loader.dataset)
        val_accuracy = accuracy_score(val_targets, val_preds)

        print(f"Epoch {epoch + 1}/{epochs}: Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    test_preds, test_targets = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            pixel_values = images

            # Forward pass
            outputs = model(pixel_values)

            # Store predictions and targets
            test_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            test_targets.extend(labels.cpu().numpy())

    # Calculate metrics
    precision = precision_score(test_targets, test_preds, average='weighted')
    recall = recall_score(test_targets, test_preds, average='weighted')
    f1 = f1_score(test_targets, test_preds, average='weighted')
    accuracy = accuracy_score(test_targets, test_preds)

    return accuracy, precision, recall, f1

criterion = nn.CrossEntropyLoss()
# 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []

for fold, (train_indices, val_indices) in enumerate(kf.split(dataset)):
    print(f"\nFold {fold + 1}")
    
    # Create data loaders for this fold
    train_subset = Subset(dataset, train_indices)
    val_subset = Subset(dataset, val_indices)
    
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
    
    # Reinitialize model for each fold
    model = SwinClassifier(base_model).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Train the model on this fold
    train_model(model, train_loader, val_loader, criterion, optimizer, epochs)
    
    # Evaluate on the validation set
    accuracy, precision, recall, f1 = evaluate_model(model, val_loader)
    print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
    
    fold_results.append((accuracy, precision, recall, f1))

# Calculate average metrics across folds
average_results = np.mean(fold_results, axis=0)
print("\n5-Fold Cross-Validation Results:")
print(f"Average Accuracy: {average_results[0]:.4f}")
print(f"Average Precision: {average_results[1]:.4f}")
print(f"Average Recall: {average_results[2]:.4f}")
print(f"Average F1 Score: {average_results[3]:.4f}")


Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([2, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Fold 1
Epoch 1/30: Train Loss: 0.3378, Train Accuracy: 0.8523, Val Loss: 0.1316, Val Accuracy: 0.9593
Epoch 2/30: Train Loss: 0.1282, Train Accuracy: 0.9538, Val Loss: 0.0657, Val Accuracy: 0.9828
Epoch 3/30: Train Loss: 0.0518, Train Accuracy: 0.9796, Val Loss: 0.0235, Val Accuracy: 0.9922
Epoch 4/30: Train Loss: 0.0632, Train Accuracy: 0.9785, Val Loss: 0.0859, Val Accuracy: 0.9656
Epoch 5/30: Train Loss: 0.0494, Train Accuracy: 0.9851, Val Loss: 0.0647, Val Accuracy: 0.9875
Epoch 6/30: Train Loss: 0.0514, Train Accuracy: 0.9832, Val Loss: 0.0447, Val Accuracy: 0.9906
Epoch 7/30: Train Loss: 0.0156, Train Accuracy: 0.9941, Val Loss: 0.0550, Val Accuracy: 0.9906
Epoch 8/30: Train Loss: 0.0410, Train Accuracy: 0.9843, Val Loss: 0.0559, Val Accuracy: 0.9812
Epoch 9/30: Train Loss: 0.0546, Train Accuracy: 0.9851, Val Loss: 0.0674, Val Accuracy: 0.9781
Epoch 10/30: Train Loss: 0.0327, Train Accuracy: 0.9898, Val Loss: 0.0754, Val Accuracy: 0.9875
Epoch 11/30: Train Loss: 0.0258, Train Ac